In [1]:
import numpy as np
import pandas as pd
#чтобы полный текст посмотреть
pd.set_option('display.max_colwidth', None)
#from pymorphy2 import MorphAnalyzer
# лемматизация и мешок слов
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('stopwords')
#удаляю русские стоп слова (расширить???)
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
#import spacy
#from spacy.lang.ru import Russian
lemmatizer = WordNetLemmatizer()

In [2]:
main_data = pd.DataFrame(pd.read_excel('../datasets/ai_bot_app_post.xls'))

In [3]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88182 entries, 0 to 88181
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88182 non-null  int64 
 1   telegram_id  88182 non-null  int64 
 2   text         88178 non-null  object
 3   title        88177 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.7+ MB


In [4]:
main_data = main_data.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['text', 'title'], keep='first')
main_data.info()
#ушло 11747 13%, пустые тоже вылетают 

<class 'pandas.core.frame.DataFrame'>
Index: 76435 entries, 0 to 88180
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      76435 non-null  object
 1   telegram_id  76435 non-null  object
 2   text         76435 non-null  object
 3   title        76435 non-null  object
dtypes: object(4)
memory usage: 2.9+ MB


In [5]:
#ch_nan = main_data['text'].isnull().values.any()
#print(ch_nan)

In [6]:
#случайным образом выбираем 10 % новостей
main_data = main_data.sample(frac=0.1)
main_data.head(1)

,user_id,telegram_id,text,title
8555,8677,18547,по данным «базы» следов теракта на месте пожара на новочеркасской грэс не обнаружено.\n\nна данный момент известно о трех пострадавших. у них ожоги рук и лица.,по данным «базы» следов теракта на месте пожара на новочеркасской грэс не обнаружено


In [7]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7644 entries, 8555 to 25312
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      7644 non-null   object
 1   telegram_id  7644 non-null   object
 2   text         7644 non-null   object
 3   title        7644 non-null   object
dtypes: object(4)
memory usage: 298.6+ KB


In [8]:
#main_data = main_data.iloc[0:30]

In [9]:
#df_new.head(1)

In [10]:
main_data['text_clean'] = main_data['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
#df_new['text_clean'] = df_new['text_clean'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
#df_new['text_clean'].head(3)

In [11]:
main_data['text_clean'] = main_data['text_clean'].str.replace('предложить новость', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('подписаться', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('SUBSCRIBE', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('НАСТОЯЩИЙ МАТЕРИАЛ (ИНФОРМАЦИЯ) ПРОИЗВЕДЕН, РАСПРОСТРАНЕН И (ИЛИ) НАПРАВЛЕН ИНОСТРАННЫМ АГЕНТОМ', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('ЛИБО КАСАЕТСЯ ДЕЯТЕЛЬНОСТИ ИНОСТРАННОГО АГЕНТА', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('\n', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('\t', ' ')
main_data['text_clean'] = main_data['text_clean'].str.replace(' {2,}', ' ', regex=True)
main_data['text_clean'] = main_data['text_clean'].str.strip()
#df_new.head(3)   нужно ли это внизу?????
main_data['text_clean'] = main_data['text_clean'].str.replace('\d+', '', regex=True)

In [12]:
main_data['text_clean'] = main_data['text_clean'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()
#df_new['text_clean'] = df_new['text'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()
#df_new.head(3)

In [13]:
main_data.head(1)

,user_id,telegram_id,text,title,text_clean
8555,8677,18547,по данным «базы» следов теракта на месте пожара на новочеркасской грэс не обнаружено.\n\nна данный момент известно о трех пострадавших. у них ожоги рук и лица.,по данным «базы» следов теракта на месте пожара на новочеркасской грэс не обнаружено,по данным базы следов теракта на месте пожара на новочеркасской грэс не обнаружено на данный момент известно о трех пострадавших у них ожоги рук и лица


In [16]:
import re
def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    #tokens = re.split('\W+', text)
    review = [word for word in tokens if word not in russian_stopwords]
    review = [nltk.WordNetLemmatize(word) for word in review]
    return review

main_data['text_clean'] = main_data['text_clean'].apply(lambda x: remove_stop_words(str(x)))

AttributeError: module 'nltk' has no attribute 'WordNetLemmatize'

In [ ]:
main_data.head(1)

In [51]:
#nlp = spacy.load('ru_core_news_md')
def tokenize_n_normalize(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(sent, pat)]

main_data['text_lemma'] = main_data['text_clean'].map(lambda x: " ".join(tokenize_n_normalize(str(x))))

In [27]:
main_data.head(1)

,user_id,telegram_id,text,title,text_clean
21944,22078,128182,"путин: россия будет форсировать массовый выпуск бпла, хорошо зарекомендовавших себя в боевых условиях.","путин: россия будет форсировать массовый выпуск бпла, хорошо зарекомендовавших себя в боевых условиях.","[путин, россия, форсировать, массовый, выпуск, бпла, зарекомендовавших, боевых, условиях, ]"


In [28]:
main_data.to_csv('../datasets/ready_to_work_try1.csv')

In [18]:
main_data.to_excel("../datasets/ready_to_work.xlsx")